## Обзор цикла статей по архитектуре YOLO

Выбрал для обзора статьи, касающиеся YOLO v1-v3:
* **[YOLO v1]** You Only Look Once: Unified, Real-Time Object Detection (https://arxiv.org/pdf/1506.02640.pdf)
* **[YOLO v2]** YOLO9000: Better, Faster, Stronger (https://arxiv.org/pdf/1612.08242.pdf)
* **[YOLO v3]** YOLOv3: An Incremental Improvement (https://pjreddie.com/media/files/papers/YOLOv3.pdf)

### YOLO v1

Впервые об архитектуре YOLO было упомянуто в 2016 году. Как утверждают авторы статьи, был придуман новый подход к детектированию объектов на картинке, работающий значительно быстрее всех своих аналогов (YOLO - 45 кадров/с, fast YOLO 155 кадров/с, при измерениях использовался Titan X GPU). Точность при этом в 2 раза превосходила другие существующие на тот момент и работающие в реальном времени детекторы.
  
Недостатки существовавших на тот момент аналогов заключались в том, что детектирование объектов происходило в несколько этапов: либо использовался метод скользящего окна (sliding window), когда классификатор запускался по очереди на различных участках изображения, пытаясь обнаружить известные объекты (DPM), либо, в относительно быстром R-CNN, сначала предсказывались возможные расположения объектов (bounding boxes) и после запускался классификатор для определения типа объекта.  
<img src="figure_1.png" style="width: 500px;">
  
YOLO (от You only look once) предлагает новый подход, когда за один проход налету предсказываются и расположения объектов и их метки. Нейросеть изучает изображение целиком. Как **преимущество** выделяются **скорость**, **обработка изображения целиком**, что ведет к значительно меньшему количеству ошибок, например, при определении фона (R-CNN из-за того, что не видит глобальный контекст, часто допускает ошибки подобного рода). Также утверждается, что YOLO намного **более устойчива при детектировании новых и неизвестных объектов.**
  
Как **недостаток** выделяется **точность**. Так как bounding box является частью предсказания, иногда нейросеть не совсем точно определяет границы объекта, особенно это заметно на небольших предметах.

#### Детектирование

Разработчики YOLO объединили различные компоненты других object detection сетей в одну нейросеть.  
Технически, YOLO разделяет изображение на сетку S x S. Если центр объекта попадает в одну из клеток, то эта клетка отвечает за определение объекта.  
В каждой клетке предсказывается B bounding boxes и для каждого бокса предсказываются вероятности, что в них, во первых, содержится объект, во вторых, насколько точно этот объект выделен. Каждый bounding box содержит 5 предсказаний: координаты центра (x, y), ширину и высоту и метрику IOU.  
В каждой клетке также предсказывается C условных вероятностей классов, $\operatorname{Pr}\left(\text { Class }_{i} | \text { Object }\right)$. Каждая клетка содержит только 1 объект (вместо того, чтобы предсказывать наличие объекта в каждом bounding box)
<img src="figure_2.png" style="width: 500px;">  
Таким образом на выходе получается тензор размером $S \times S \times(B * 5+C)$

#### Строение сети

В основе YOLO лежит GoogLeNet для классификации изображений. Сеть имеет 24 сверточных слоя, за которыми следуют 2 полносвязных. Вместо модуля inception из GoogLeNet используются свертки размером 1х1 за которыми следует сверточной слой с размером 3х3. Такая же архитектура использовалась.  
Также была разработана облегченная версия fast YOLO с 9 сверточными слоями вместо 24 и меньшим количеством фильтров.  
<img src="figure_3.png" style="width: 700px;">  
Далее в статье приводятся сведения о тренировке сети и возможных комбинациях с другими сетями для увеличения точности. Также была выделена проблема, что YOLO из-за специфики определения объектов на изображении плохо справляется с группами малых объектов, например, косяками птиц. Тем не менее констатируется, что среди детекторов, работающих в реальном времени, YOLO является наиболее точной и быстрой.
<img src="figure_4.png" style="width: 300px;">

### YOLO9000

Новая версия YOLO была представлена в 2017 году. Создатели расширили список классов, детектируемых нейросетью, до 9000, сохранив при этом возможность работы в реальном времени.  
При проектировании YOLO v2 создатели сконцентрировались на увеличении recall и улучшении локализации объектов, стараясь сохранить точность предсказаний. Было протестировано и добавлено несколько подходов:
* **Batch normalization**. Добавление batch normalization на каждый сверточный слой ведет к увеличению mAP на 2%. Также это позволило отказаться от использования dropout, не боясь переобучения
* **Классификатор высокого разрешения**. Большинство нейросетей в object detection используют предтренированные классификаторы для определения типов объектов. Как правило тренировка происходит на ImageNet с разрешением 248х248. В первой версии YOLO, был использован классификатор также обученный на изображениях формата 224х224, а затем для детектирования формат был увеличен до 448х448. Сеть должна была перестроится на более высокое разрешение. В YOLO v2 классификатор изначально был предобучен на изображениях формата 448х448. Это привело к увеличению mAP на 4%
* **dimension clusters**. Был изменен подход к предсказанию размеров bounding box, который также привел к значительному улучшению в качестве предсказаний
* **direct location prediction**. Был изменен подход к предсказанию центров объектов, который дал увеличение в 5% mAP
* **fine-grained features**. Подход заключался в добавлении сквозного слоя со слоя с разрешением 26х26, который в последствии объединялся со слоем 13х13. Такой подход позволил улучшить определение мелких объектов.
* **multi-scale training**. Создадтели хотели спроектировать нейросеть, которая была бы независима от разрешения входного изображения. Возможные размеры инпутов были расширены. Теперь можно использовать любые изображения между 320х320 и 608х608 с шагом 32. Увеличение разрешения входного изображения привело к увеличению mAP.
  
Также были предложены подходы по обучения одновременно и на классификационных и на detection датасетах. В зависимости от разметки, использовалась определенная функция потерь, что позволяло отбросить в архитектуре нейросети часть, отвечающую за detection, если датасет был размечен только для классификации.  
Еще одна инновация - использование иерархии при классификации. Вместо того, чтобы напрямую предсказывать класс, были составлены деревья слов, ведущие от более общих объектов к частным. Такой подход позволяет использовать несколько меток для одного объекта
<img src="figure_5.png" style="width: 400px;">

### YOLO v3

YOLO v3 была представлена в 2018 году и содержала некоторые небольшие доработки для улучшения предыдущих версий:
* **предсказания bounding box**. Теперь используется логистическая регрессия для предсказания размеров bounding box, а также некоторые улучшения в математическом аппарате, используемом для предсказаний размеров бокса
* **новый backbone**
  
<img src="figure_6.png" style="width: 400px;">
<img src="figure_7.png" style="width: 400px;">